In [1]:
import pandas as pd
import requests
import re
from tqdm.auto import tqdm

In [79]:
# Load overview of files
df = pd.read_csv('data/UMLFiles_List_V2.0.csv', sep=',')

# Enable tqdm for pandas
tqdm.pandas()

# Initial data cleaning

In [80]:
# Get filetypes: we cannot use image files for scraping
df['filetype'] = df['Model Link - Github'].apply(lambda x: x.split('.')[-1])

# Check which filetypes are there
print(df['filetype'].unique())

# Remove rows with #VALUE! in it
df = df[df['Model Link - Github'] != '#VALUE!']

# Remove unusable file types
df = df[df['filetype'].isin(['xmi', 'uml'])]

# The dataframe contains GitHub URLs, not the "raw" files that we need
# Function to get the raw file URL from the GitHub URL
def get_raw_file(url):
    response = requests.get(url)
    data = response.text
    targets = re.findall(r'href="(.+/raw/.+?)"', data)
    
    try: 
        return 'https://github.com' + targets[0]
    except:1
        # Page no longer exists, add tag to remove later
        return 'REMOVE'

# Do the harvesting 
df['raw_filepath'] = df['Model Link - Github'].progress_apply(lambda x: get_raw_file(x))

# Remove all (currently) non-existent file paths from the dataframe
df = df[df['raw_filepath'] != 'REMOVE']

['png' 'xmi' 'uml' 'jpg' 'svg' 'jpeg' 'gif' 'bmp' '#VALUE!']


In [81]:
# Save file for later use
df.to_csv('data/cleaned_files.csv', sep=',', index=False)

# Start wrangling

In [2]:
df = pd.read_csv('data/cleaned_files.csv', sep=',')

classes = []
attributes = []

## UML file types

In [24]:
# UML file types
umls = df[df['filetype'] == 'uml']